<a href="https://colab.research.google.com/github/brepowell/ML-Contest-Series/blob/main/MLSeriesSupervisedLearningTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To run:
Use an IDE that will allow you to open a Jupyter Notebook.

For example, use Anaconda Navigator to open Visual Studio Code.

You may need to select a kernel to run the program.

## Import Libraries

In [42]:
# This step can be done in any cell of the notebook. It does not have to be at the top.
import pandas as pd
import matplotlib as plt

## Data Information

From:
https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households

More Info:
https://innovation.ukpowernetworks.co.uk/projects/low-carbon-london

"Created 9 years ago, updated 2 years ago
Energy consumption readings for a sample of 5,567 London Households that took part in the UK Power Networks led Low Carbon London project between November 2011 and February 2014.

Readings were taken at half hourly intervals. The customers in the trial were recruited as a balanced sample representative of the Greater London population.

The dataset contains energy consumption, in kWh (per half hour), unique household identifier, date and time. The CSV file is around 10GB when unzipped and contains around 167million rows.

Within the data set are two groups of customers. The first is a sub-group, of approximately 1100 customers, who were subjected to Dynamic Time of Use (dToU) energy prices throughout the 2013 calendar year period. The tariff prices were given a day ahead via the Smart Meter IHD (In Home Display) or text message to mobile phone. Customers were issued High (67.20p/kWh), Low (3.99p/kWh) or normal (11.76p/kWh) price signals and the times of day these applied. The dates/times and the price signal schedule is availaible as part of this dataset. All non-Time of Use customers were on a flat rate tariff of 14.228pence/kWh.

The signals given were designed to be representative of the types of signal that may be used in the future to manage both high renewable generation (supply following) operation and also test the potential to use high price signals to reduce stress on local distribution grids during periods of stress.

The remaining sample of approximately 4500 customers energy consumption readings were not subject to the dToU tariff."


## Create a huge dataframe with all hourly info

In [43]:
##############################
# This takes about 2 minutes #
##############################

import glob

def readAllHourlyData():
    # Read all CSV files and concatenate them into a single DataFrame
    path = "data/archive/halfhourly_dataset/halfhourly_dataset/" 

    all_files = glob.glob(path + "block_*.csv")
    df_list = []
    for filename in all_files:
        df = pd.read_csv(filename)
        df_list.append(df)
    full_df = pd.concat(df_list)

    return full_df

londonData = readAllHourlyData()
londonData.shape

C:\Users\brely\AppData\Local\Temp\ipykernel_26776\3336080981.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_26776\3336080981.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_26776\3336080981.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_26776\3336080981.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\AppData\Local\Temp\ipykernel_26776\3336080981.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\brely\

MemoryError: Unable to allocate 1.25 GiB for an array with shape (167817021,) and data type int64

In [ ]:
londonData.head()

,LCLid,tstp,energy(kWh/hh)
0,MAC000002,2012-10-12 00:30:00.0000000,0
1,MAC000002,2012-10-12 01:00:00.0000000,0
2,MAC000002,2012-10-12 01:30:00.0000000,0
3,MAC000002,2012-10-12 02:00:00.0000000,0
4,MAC000002,2012-10-12 02:30:00.0000000,0


## Select Features

In [ ]:
# What features do I want?
# Weekends / Weekdays
# How many customers were represented on that day? - Might be needed for normalization
# Dip in customers? - Could be a sign of an outage
# Blackout signature - mark hours before 0 power as signature days?
# NaN's - boolean yes or no if there were NaN's in that row. Replace NaN with 0.
# Weather Features - add later to see if improves

### NaN

In [ ]:
import pandas as pd

londonData["isNA"] = londonData["energy(kWh/hh)"].isna()
print("Number of NaN values: \n{}".format(londonData["isNA"].value_counts()))
londonData.head()


Number of NaN values: False    167817021
Name: isNA, dtype: int64


,LCLid,energy(kWh/hh),year,month,day,hour,minute,second,dayNumber,unixTimeStamp,weekend,dayString,isNA
0,MAC000002,0,2012,10,12,0,30,0,4,1350001800,False,2012-10-12,False
1,MAC000002,0,2012,10,12,1,0,0,4,1350003600,False,2012-10-12,False
2,MAC000002,0,2012,10,12,1,30,0,4,1350005400,False,2012-10-12,False
3,MAC000002,0,2012,10,12,2,0,0,4,1350007200,False,2012-10-12,False
4,MAC000002,0,2012,10,12,2,30,0,4,1350009000,False,2012-10-12,False


,LCLid,energy(kWh/hh),year,month,day,hour,minute,second,dayNumber,unixTimeStamp,weekend,dayString
isNA,,,,,,,,,,,,
False,167817021,167817021,167817021,167817021,167817021,167817021,167817021,167817021,167817021,167817021,167817021,167817021


### Day Information

In [ ]:
##############################
# This takes about 2 minutes #
##############################

def extractDayInformation(data):

    # Convert 'tstp' column to datetime
    data['tstp'] = pd.to_datetime(data['tstp'])
    
    # Extract day information
    data['year'] = data['tstp'].dt.year
    data['month'] = data['tstp'].dt.month
    data['day'] = data['tstp'].dt.day
    data['hour'] = data['tstp'].dt.hour
    data['minute'] = data['tstp'].dt.minute
    data['second'] = data['tstp'].dt.second
    data['dayNumber'] = data['tstp'].dt.dayofweek
    data['unixTimeStamp'] = (data['tstp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

    # Get rid of the tstp column
    data = data.drop(['tstp'], axis=1)
    
    return data

londonData = extractDayInformation(londonData)
londonData.head()


,LCLid,energy(kWh/hh),year,month,day,hour,minute,second,dayNumber,unixTimeStamp
0,MAC000002,0,2012,10,12,0,30,0,4,1350001800
1,MAC000002,0,2012,10,12,1,0,0,4,1350003600
2,MAC000002,0,2012,10,12,1,30,0,4,1350005400
3,MAC000002,0,2012,10,12,2,0,0,4,1350007200
4,MAC000002,0,2012,10,12,2,30,0,4,1350009000


In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.dayofweek.html
# 0 is Monday; 6 is Sunday; 5 is Saturday
londonData["weekend"] = londonData["dayNumber"] > 4

In [ ]:
londonData.head()
path = "Data/pickles/"
londonData.to_pickle(path + "londonData.pkl")

### Customer Count Per Day

In [ ]:
###############################
# This takes about 35 seconds #
###############################

import pandas as pd
path = "Data/pickles/"
londonData = pd.read_pickle(path + "londonData.pkl")

In [ ]:
londonData['dayString'] = londonData['year'].astype(str) + "-" + londonData['month'].astype(str) + "-" + londonData['day'].astype(str)
londonData.head()

,LCLid,energy(kWh/hh),year,month,day,hour,minute,second,dayNumber,unixTimeStamp,weekend,dayString
0,MAC000002,0,2012,10,12,0,30,0,4,1350001800,False,2012-10-12
1,MAC000002,0,2012,10,12,1,0,0,4,1350003600,False,2012-10-12
2,MAC000002,0,2012,10,12,1,30,0,4,1350005400,False,2012-10-12
3,MAC000002,0,2012,10,12,2,0,0,4,1350007200,False,2012-10-12
4,MAC000002,0,2012,10,12,2,30,0,4,1350009000,False,2012-10-12


In [ ]:
londonData.shape

(167817021, 12)

In [ ]:
##############################
# This takes about 2 minutes #
##############################
# Weekends / Weekdays
# How many customers were represented on that day? - Might be needed for normalization
# Dip in customers? - Could be a sign of an outage
# Blackout signature - mark hours before 0 power as signature days?
# NaN's - boolean yes or no if there were NaN's in that row. Replace NaN with 0.
# Weather Features - add later to see if improves


allDays = londonData.groupby("year").count()
allDays = allDays.groupby("month").count()
allDays = allDays.groupby("day").count()
allDays.head()

,LCLid,energy(kWh/hh),hour,minute,second,dayNumber,unixTimeStamp,weekend,dayString
day,,,,,,,,,
1,4,4,4,4,4,4,4,4,4


In [ ]:
import matplotlib.dates as mdates

def plotCustomersPerDay(days):

    # Plotting the data
    plt.figure(figsize=(10, 6))
    plt.plot(days.index, days)

    # Set x-axis ticks to show once a month
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

    plt.xlabel('Days')
    plt.ylabel('Customers Per Day')
    plt.title('Customers Per Day -- WHOLE SPAN')
    plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility
    plt.tight_layout()
    plt.show()

plotCustomersPerDay(allDays)

AttributeError: module 'matplotlib' has no attribute 'figure'

## Visualize the features

In [ ]:
#https://www.tensorflow.org/tutorials/structured_data/time_series

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
1435,MAC000150,2011-11-24,0.158,0.204,0.158,0.149,0.165,0.135,0.137,0.094,...,0.222,0.376,1.069,0.350,0.671,0.434,0.711,0.198,0.228,0.228
23891,MAC004919,2012-01-22,0.098,0.051,0.032,0.066,0.026,0.025,0.062,0.039,...,0.549,0.461,0.309,0.310,0.073,0.095,0.086,0.063,0.088,0.050
19711,MAC005041,2012-02-19,0.392,5.556,5.432,5.393,4.893,4.648,4.161,2.961,...,0.405,0.431,0.457,0.393,0.365,0.392,0.407,0.354,0.991,0.921
3987,MAC000214,2012-03-11,0.096,0.062,0.044,0.096,0.066,0.043,0.091,0.069,...,0.295,0.253,0.182,0.169,0.265,0.305,0.294,0.285,0.221,0.218
22271,MAC004391,2012-03-28,0.343,0.169,0.190,0.175,0.123,0.183,0.142,0.295,...,0.607,0.488,0.256,0.401,0.804,0.671,0.956,0.475,0.176,0.260


# STEP 2: FEATURE ENGINEERING / DATA CLEANING / PRE-PROCESSING TECHNIQUES

## Fix the data imbalance

## Seperate features (x) from labels (y)

## Feature Reduction

## Normalize / Scale the Data

## Investigate Variance or Feature Importance

# STEP 3: MODEL TRAINING & BUILDING


## Split into Train and Test data

## Use a Model

## Perform a Hyperparameter Search

## Save the best model

# STEP 4: EVALUATE THE MODEL

## Look at Metrics - Ex) Precision, Recall, F1 score

## Plot a Confusion Matrix

# STEP 5: DEPLOY MODEL